In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import cross_val_score
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

In [2]:
train = pd.read_csv('train.csv')

In [3]:
test = pd.read_csv('test.csv')

In [4]:
train.shape

(2952, 15)

> 데이터의 수 자체가 적음, 전처리가 중요

In [5]:
train.isna().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              0
임대보증금                           569
임대료                             569
도보 10분거리 내 지하철역 수(환승노선 수 반영)    211
도보 10분거리 내 버스정류장 수                4
단지내주차면수                           0
등록차량수                             0
dtype: int64

> 총 4개의 column에서 결측치 발생

In [6]:
test.isna().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              2
임대보증금                           180
임대료                             180
도보 10분거리 내 지하철역 수(환승노선 수 반영)     42
도보 10분거리 내 버스정류장 수                0
단지내주차면수                           0
dtype: int64

> 총 4개의 column에서 결측치발생 

> train에서는 결측치가 발생하지않은 column : `자격유형`

In [7]:
train_nunique = train.nunique()
train_nunique

단지코드                            423
총세대수                            351
임대건물구분                            2
지역                               16
공급유형                             10
전용면적                            679
전용면적별세대수                        403
공가수                              48
자격유형                             15
임대보증금                           957
임대료                             995
도보 10분거리 내 지하철역 수(환승노선 수 반영)      4
도보 10분거리 내 버스정류장 수               17
단지내주차면수                         355
등록차량수                           354
dtype: int64

In [8]:
train_type_nunique = pd.DataFrame({
    'Data Type': train.dtypes,
    'Number of Unique Values': train.nunique()
})

train_type_nunique

,Data Type,Number of Unique Values
단지코드,object,423
총세대수,int64,351
임대건물구분,object,2
지역,object,16
공급유형,object,10
전용면적,float64,679
전용면적별세대수,int64,403
공가수,float64,48
자격유형,object,15
임대보증금,object,957


> object이면서 nunique가 많은 값 : `공급유형`, `자격유형`, `지역`

In [9]:
# 지역 data 합치기

train.loc[train.지역.isin(['경기도','서울특별시']), '지역'] = '수도권(서울/경기)'
train.loc[train.지역.isin(['경상남도','울산광역시','부산광역시','대구광역시','경상북도']), '지역'] = '경상도'
train.loc[train.지역.isin(['전라남도','광주광역시','전라북도']), '지역'] = '전라도'
train.loc[train.지역.isin(['충청남도','충청북도','대전광역시','세종특별자치시']), '지역'] = '충청도'

test.loc[test.지역.isin(['경기도','서울특별시']), '지역'] = '수도권(서울/경기)'
test.loc[test.지역.isin(['경상남도','울산광역시','부산광역시','대구광역시','경상북도']), '지역'] = '경상도'
test.loc[test.지역.isin(['전라남도','광주광역시','전라북도']), '지역'] = '전라도'
test.loc[test.지역.isin(['충청남도','충청북도','대전광역시','세종특별자치시']), '지역'] = '충청도'

In [10]:
train['공급유형'].unique()

array(['국민임대', '공공임대(50년)', '영구임대', '임대상가', '공공임대(10년)', '공공임대(분납)',
       '장기전세', '공공분양', '행복주택', '공공임대(5년)'], dtype=object)

In [11]:
# train, test column rename : column명이 긴 것을 간결화
train.rename(columns={'도보 10분거리 내 지하철역 수(환승노선 수 반영)': '지하철수',
                      '도보 10분거리 내 버스정류장 수': '버스수'}, inplace=True)
test.rename(columns={'도보 10분거리 내 지하철역 수(환승노선 수 반영)': '지하철수',
                     '도보 10분거리 내 버스정류장 수': '버스수'}, inplace=True)


# 결측치 통일 : '임대료','임대보증금' 컬럼에서 '-' 값을 NaN으로 변경

for col in ['임대료', '임대보증금']:
    train[col] = train[col].replace('-', np.nan).astype(float)
    test[col] = test[col].replace('-', np.nan).astype(float)

# 해당 컬럼을 float64 타입으로 변환
train['임대료'] = train['임대료'].astype(float)
train['임대보증금'] = train['임대보증금'].astype(float)

# 해당 컬럼을 float64 타입으로 변환
test['임대료'] = test['임대료'].astype(float)
test['임대보증금'] = test['임대보증금'].astype(float)

# 버스수, 지하철수의 결측치를 0으로 채움
# <위의 근거를 써주세요!>

train['버스수'].fillna(0, inplace=True)
train['지하철수'].fillna(0, inplace=True)

#C2411 자격유형 → 'A'
# <위의 근거를 써주세요!>
test[test['단지코드']=='C2411']['자격유형'].fillna('A')

#C2253 자격유형 → 'C'
# <위의 근거를 써주세요!>
test[test['단지코드']=='C2253']['자격유형'].fillna('C')

train.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철수,버스수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상도,국민임대,39.72,134,38.0,A,15667000.0,103680.0,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상도,국민임대,39.72,15,38.0,A,15667000.0,103680.0,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상도,국민임대,51.93,385,38.0,A,27304000.0,184330.0,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상도,국민임대,51.93,15,38.0,A,27304000.0,184330.0,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상도,국민임대,51.93,41,38.0,A,27304000.0,184330.0,0.0,3.0,1425.0,1015.0


In [12]:
# Filter rows where '임대료' or '임대보증금' is NaN
missing_data = train[train['임대료'].isna() | train['임대보증금'].isna()]

# Display statistical summary and the first few rows of the missing data
summary = missing_data.describe(include='all')
summary


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철수,버스수,단지내주차면수,등록차량수
count,590,590.000000,590,590,590,590.000000,590.000000,590.000000,590,9.000000e+00,0.0,590.000000,590.000000,590.000000,590.000000
unique,39,NaN,2,5,5,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
top,C1439,NaN,상가,경상도,임대상가,NaN,NaN,NaN,D,NaN,NaN,NaN,NaN,NaN,NaN
freq,45,NaN,562,325,562,NaN,NaN,NaN,569,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,1236.922034,NaN,NaN,NaN,45.542356,7.984746,7.886441,NaN,1.842430e+08,NaN,0.288136,3.679661,313.910169,213.735593
std,NaN,652.934635,NaN,NaN,NaN,66.314353,45.403471,10.514231,NaN,3.962353e+07,NaN,0.453279,2.308343,293.986239,308.756185
min,NaN,370.000000,NaN,NaN,NaN,12.620000,1.000000,0.000000,NaN,8.744400e+07,NaN,0.000000,1.000000,65.000000,31.000000
25%,NaN,657.000000,NaN,NaN,NaN,26.250000,1.000000,1.000000,NaN,1.748880e+08,NaN,0.000000,2.000000,153.000000,94.000000
50%,NaN,1005.000000,NaN,NaN,NaN,31.840000,1.000000,2.000000,NaN,1.945620e+08,NaN,0.000000,3.000000,217.000000,127.000000
75%,NaN,1755.000000,NaN,NaN,NaN,37.950000,1.000000,9.000000,NaN,2.138630e+08,NaN,1.000000,4.000000,351.000000,189.000000


> `임대건물구분`에서 대부분의 행이 '상가'로 분류되어 있습니다.

> `공급유형`은 대부분 '임대상가'입니다.

> `자격유형`은 모두 'D'입니다.


In [13]:
print("임대건물구분이 상가인 것의 임대료",train[train['임대건물구분'] == '상가']['임대료'].unique())
print("임대건물구분이 상가인 것의 임대보증금",train[train['임대건물구분'] == '상가']['임대보증금'].unique())
print("공급유형이 임대상가인 것의 임대료",train[train['공급유형']=='임대상가']['임대료'].unique())
print("공급유형이 임대상가인 것의 임대보증금",train[train['공급유형']=='임대상가']['임대보증금'].unique())
print("자격유형이 D인 것의 임대료",train[train['자격유형'] == 'D']['임대료'].unique())
print("자격유형이 D인 것의 임대보증금",train[train['자격유형'] == 'D']['임대보증금'].unique())

임대건물구분이 상가인 것의 임대료 [nan]
임대건물구분이 상가인 것의 임대보증금 [nan]
공급유형이 임대상가인 것의 임대료 [nan]
공급유형이 임대상가인 것의 임대보증금 [nan]
자격유형이 D인 것의 임대료 [nan]
자격유형이 D인 것의 임대보증금 [nan]


> 해당하는 조건을 해봤으나 모두 nan값이라 유의미한 결과를 도출할 수 없음

> data type이 object인 것을 모두 인코딩(원-핫 인코딩 사용)

In [14]:
# One-hot encode the object columns (except '단지코드')

# Identify the object columns to be encoded
object_columns = train.select_dtypes(include=['object']).columns.tolist()
object_columns.remove('단지코드')  # Exclude '단지코드'

# One-hot encode
train_encoded = pd.get_dummies(train, columns=object_columns)
test_encoded = pd.get_dummies(test, columns=object_columns)

# Display the first few rows of the encoded data
train_encoded.head()
train_encoded.columns

Index(['단지코드', '총세대수', '전용면적', '전용면적별세대수', '공가수', '임대보증금', '임대료', '지하철수',
       '버스수', '단지내주차면수', '등록차량수', '임대건물구분_상가', '임대건물구분_아파트', '지역_강원도',
       '지역_경상도', '지역_수도권(서울/경기)', '지역_전라도', '지역_제주특별자치도', '지역_충청도',
       '공급유형_공공분양', '공급유형_공공임대(10년)', '공급유형_공공임대(50년)', '공급유형_공공임대(5년)',
       '공급유형_공공임대(분납)', '공급유형_국민임대', '공급유형_영구임대', '공급유형_임대상가', '공급유형_장기전세',
       '공급유형_행복주택', '자격유형_A', '자격유형_B', '자격유형_C', '자격유형_D', '자격유형_E', '자격유형_F',
       '자격유형_G', '자격유형_H', '자격유형_I', '자격유형_J', '자격유형_K', '자격유형_L', '자격유형_M',
       '자격유형_N', '자격유형_O'],
      dtype='object')

In [15]:
train_encoded.dtypes

단지코드               object
총세대수                int64
전용면적              float64
전용면적별세대수            int64
공가수               float64
임대보증금             float64
임대료               float64
지하철수              float64
버스수               float64
단지내주차면수           float64
등록차량수             float64
임대건물구분_상가            bool
임대건물구분_아파트           bool
지역_강원도               bool
지역_경상도               bool
지역_수도권(서울/경기)        bool
지역_전라도               bool
지역_제주특별자치도           bool
지역_충청도               bool
공급유형_공공분양            bool
공급유형_공공임대(10년)       bool
공급유형_공공임대(50년)       bool
공급유형_공공임대(5년)        bool
공급유형_공공임대(분납)        bool
공급유형_국민임대            bool
공급유형_영구임대            bool
공급유형_임대상가            bool
공급유형_장기전세            bool
공급유형_행복주택            bool
자격유형_A               bool
자격유형_B               bool
자격유형_C               bool
자격유형_D               bool
자격유형_E               bool
자격유형_F               bool
자격유형_G               bool
자격유형_H               bool
자격유형_I               bool
자격유형_J      

In [16]:
train_encoded

,단지코드,총세대수,전용면적,전용면적별세대수,공가수,임대보증금,임대료,지하철수,버스수,단지내주차면수,...,자격유형_F,자격유형_G,자격유형_H,자격유형_I,자격유형_J,자격유형_K,자격유형_L,자격유형_M,자격유형_N,자격유형_O
0,C2483,900,39.72,134,38.0,15667000.0,103680.0,0.0,3.0,1425.0,...,False,False,False,False,False,False,False,False,False,False
1,C2483,900,39.72,15,38.0,15667000.0,103680.0,0.0,3.0,1425.0,...,False,False,False,False,False,False,False,False,False,False
2,C2483,900,51.93,385,38.0,27304000.0,184330.0,0.0,3.0,1425.0,...,False,False,False,False,False,False,False,False,False,False
3,C2483,900,51.93,15,38.0,27304000.0,184330.0,0.0,3.0,1425.0,...,False,False,False,False,False,False,False,False,False,False
4,C2483,900,51.93,41,38.0,27304000.0,184330.0,0.0,3.0,1425.0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,C2532,239,49.20,19,7.0,11346000.0,116090.0,0.0,1.0,166.0,...,False,False,False,False,False,False,False,False,False,False
2948,C2532,239,51.08,34,7.0,14005000.0,142310.0,0.0,1.0,166.0,...,False,False,False,False,False,False,False,False,False,False
2949,C2532,239,51.73,34,7.0,14005000.0,142310.0,0.0,1.0,166.0,...,False,False,False,False,False,False,False,False,False,False
2950,C2532,239,51.96,114,7.0,14005000.0,142310.0,0.0,1.0,166.0,...,False,False,False,False,False,False,False,False,False,False


In [17]:
train_encoded = train_encoded.drop(columns= ['임대보증금', '임대료'])
test_encoded = test_encoded.drop(columns= ['임대보증금', '임대료'])

In [19]:
train_encoded

In [18]:
from sklearn.model_selection import train_test_split


# X와 y 분리
X = train_encoded.drop(['단지코드','등록차량수'], axis=1)  # 'target'은 예측하고자 하는 타겟 변수입니다.
y = train_encoded['등록차량수']

# 학습 데이터와 검증 데이터로 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 학습
lasso = Lasso(alpha=1.0)
ridge = Ridge(alpha=1.0)
elastic_net = ElasticNet(alpha=1.0, l1_ratio=0.5)

lasso.fit(X_train, y_train)
ridge.fit(X_train, y_train)
elastic_net.fit(X_train, y_train)

# 예측
lasso_pred = lasso.predict(X_val)
ridge_pred = ridge.predict(X_val)
elastic_net_pred = elastic_net.predict(X_val)

from sklearn.metrics import mean_absolute_error

# 성능 평가 (MAE)
lasso_mae = mean_absolute_error(y_val, lasso_pred)
ridge_mae = mean_absolute_error(y_val, ridge_pred)
elastic_net_mae = mean_absolute_error(y_val, elastic_net_pred)

min_mae = min(lasso_mae, ridge_mae, elastic_net_mae)

if min_mae == lasso_mae:
    best_model = lasso
    model_name = "Lasso"
elif min_mae == ridge_mae:
    best_model = ridge
    model_name = "Ridge"
else:
    best_model = elastic_net
    model_name = "ElasticNet"

print(f"Best Model: {model_name} with MAE: {min_mae}")



AttributeError: 'NoneType' object has no attribute 'drop'

In [ ]:
# X와 y 분리
X = train_non_store_data.drop(['단지코드','등록차량수'], axis=1)  # 'target'은 예측하고자 하는 타겟 변수입니다.
y = train_non_store_data['등록차량수']

# 학습 데이터와 검증 데이터로 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 학습
lasso = Lasso(alpha=1.0)
ridge = Ridge(alpha=1.0)
elastic_net = ElasticNet(alpha=1.0, l1_ratio=0.5)

lasso.fit(X_train, y_train)
ridge.fit(X_train, y_train)
elastic_net.fit(X_train, y_train)

# 예측
lasso_pred = lasso.predict(X_val)
ridge_pred = ridge.predict(X_val)
elastic_net_pred = elastic_net.predict(X_val)

from sklearn.metrics import mean_absolute_error

# 성능 평가 (MAE)
lasso_mae = mean_absolute_error(y_val, lasso_pred)
ridge_mae = mean_absolute_error(y_val, ridge_pred)
elastic_net_mae = mean_absolute_error(y_val, elastic_net_pred)

min_mae = min(lasso_mae, ridge_mae, elastic_net_mae)

if min_mae == lasso_mae:
    best_model = lasso
    model_name = "Lasso"
elif min_mae == ridge_mae:
    best_model = ridge
    model_name = "Ridge"
else:
    best_model = elastic_net
    model_name = "ElasticNet"

print(f"Best Model: {model_name} with MAE: {min_mae}")

